In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import torch
import pickle
import json
from torchvision.datasets import CIFAR10, CIFAR100, SVHN, ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm
from pathlib import Path
from PIL import Image
from matplotlib import pyplot as plt
from collections import Counter
from helpers import *
from explore import *
from sklearn import metrics
from itertools import product
from pprint import pprint
pd.set_option('display.max_rows', 1000)

In [2]:
cifar10_class_names = ["airplane", "automobile","bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [3]:
def softmax_t(logits, temp=1):
    logits = logits/temp
    _max = np.expand_dims(np.max(logits, axis=-1), axis=-1)
    probs = np.exp(logits - _max)
    _sum = np.expand_dims(np.sum(probs, axis=-1), axis=-1)
    return probs/_sum

def cluster_purity(kmeans_targets, in_targets):
    k_classes = np.unique(kmeans_targets).astype(int)
    k_class_idx = [np.nonzero(np.equal(cls_, kmeans_targets)) for cls_ in k_classes]
    in_classes_in_k_clstr = [in_targets[idx] for idx in k_class_idx]
    purity_list = []

    for cluster_k in in_classes_in_k_clstr:
        unique, counts = np.unique(cluster_k, return_counts=True)
        purity_list.append(np.round(np.asarray(counts).max()/len(cluster_k), 5))

    return purity_list

## Load Model

In [4]:
encoder_type = "key"
output_layer = "avg_pool"

## Load IID

In [ ]:
# load id
iid = "CIFAR10"
prefix = f"{iid}_{encoder_type}_{output_layer}"
prefix

In [ ]:
id_train = load_features("../cache", f"{prefix}_id_train.npy")
id_test = load_features("../cache", f"{prefix}_id_test.npy")
id_train_targ = load_features("../cache", f"{prefix}_id_train_targ.npy")
id_test_targ = load_features("../cache", f"{prefix}_id_test_targ.npy")

## Load OOD

In [ ]:
# load ood
oods = ["CIFAR100"]

def get_ood_feats(ood):
    prefix = f"{ood}_{encoder_type}_{output_layer}"
    print(f"Using {prefix}")
    ood_train = load_features("../cache", f"{prefix}_ood_train.npy")
    ood_test = load_features("../cache",  f"{prefix}_ood_test.npy")
    return ood_train, ood_test

## Train Features

In [ ]:
#train_linear_model(id_train, id_train_targ, id_test,  id_test_targ, 3)

## Get Prototypes

In [ ]:
get_prototypes = False

In [ ]:
if get_prototypes:
    num_cluster = 768
    id_im2cluster, id_prototypes, id_density = run_clustering(norm_feats(id_train), num_cluster)

## Perform OOD detection

In [ ]:
#clus = 400
#pca_com = 0
#oe = OodEvaluator(norm_feats(id_train), norm_feats(id_test), id_train_targ, id_test_targ,
#                num_clusters = clus,
#                pca_com = pca_com,
#                cluster_method = "kmeans",
#                means = None,
#                im2cluster = None,
#                clip = 0,
#                clip_metric = None)

ood_results = []
clusters = [10, 15, 20, 30]
pcas = [10]
#for clip in np.linspace(0, 0.9, 10):
for ood, clus, pca_com in product(oods, clusters, pcas):
    ood_train, ood_test = get_ood_feats(ood)
    oe = OodEvaluator(norm_feats(id_train), norm_feats(id_test), id_train_targ, id_test_targ,
                    num_clusters = clus,
                    pca_com = pca_com,
                    cluster_method = "gmm",
                    means = None,
                    im2cluster = None,
                    clip = 0,
                    clip_metric = "cosine")
    
    for met in ["mahalanobis", "cosine"]:
        if met == "mahalanobis" and pca_com == 0:
            continue
            
        oe(norm_feats(ood_test), met)
        oe.get_scores()
        oe.get_auroc(sklean_auroc=False)
        auroc, tnr =  oe.auroc, oe.tnr_at_tpr95
        oe.get_auroc(sklean_auroc=True)
        sklearn_auroc = oe.auroc
        o_pred = oe.out_pred_score
        i_pred = oe.test_pred_score

        ood_results.append({"ood":ood, "cluster": clus, "pca": pca_com,  "metric": met, "auroc": auroc, \
                            "tnr@tpr95": tnr, "sklearn-auroc":sklearn_auroc, \
                           "o_pred": o_pred, "i_pred": i_pred, "gmm_results":oe.gmm_results 
                            
                           })
        

In [ ]:
def plot_cluster_assignment(oe, fsize = (5, 10)):
    fig = plt.figure(figsize=fsize)
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)
    ax1.title.set_text('Train')
    a = Counter(oe.train_pred_label)
    k, v = zip(*sorted(a.items(), key=lambda x:x[0]))
    ax1.bar(k, v)
    ax1.grid()
    ax2.title.set_text('Test')
    b = Counter(oe.test_pred_label)
    k, v = zip(*sorted(b.items(), key=lambda x:x[0]))
    ax2.bar(k, v)
    ax2.grid()
    ax3.title.set_text('OOD')
    c = Counter(oe.out_pred_label)
    k, v = zip(*sorted(c.items(), key=lambda x:x[0]))
    ax3.bar(k, v)
    ax3.grid()
    plt.show()
    
def scaled(x):
    ma = np.max(oe.train_pred_score)
    mi = np.min(oe.train_pred_score)
    return (x - mi)/(ma - mi)
    
def plot_pred_scores(oe, scale=None, scale_train=False):
        
    plt.figure(figsize=(10, 7))
    plt.hist(scaled(oe.out_pred_score) if scale and scale_train else oe.out_pred_score, alpha=0.6, label="ood")
    plt.hist(scaled(oe.test_pred_score) if scale else oe.test_pred_score, alpha=0.6, label="test id")
    plt.hist(scaled(oe.train_pred_score) if scale else oe.train_pred_score, alpha=0.2, label="train id")
    plt.legend()

In [ ]:
#plot_cluster_assignment(oe)

In [ ]:
#plot_pred_scores(oe, False)

In [ ]:
#oe.reclip(0, clip_metric=None)

In [ ]:
#oe(norm_feats(ood_test), met)

In [ ]:
#oe.get_scores(rescale=False)

In [ ]:
#oe.get_auroc()

In [ ]:
#plot_cluster_assignment(oe, fsize = (5, 10))

In [ ]:
#plot_pred_scores(oe, False, False)

In [ ]:
#oe.cov_inv[20]

## Cluster Evaluation

### Step 1  - Cluster 
- Perform Clustering with Kmeans or GMM

### Step 2 -  Extrinsic Evaluation
- homogeniety score
- complenteness score
- v measure score
- adjusted rand score
- adjusted mutal information

### Step 3 -  Intrinsic Evaluation
- silhoutte - euclidean
- silhoutte - cosine
- calinski harabasz score
- davis bouldin score

### Step 4 - AUROC
- 3 values

### Step 5 - Store Data
- store distance matrices
- store cluster assignments

In [ ]:
ood = "CIFAR100"
ood_train, ood_test = get_ood_feats(ood)
pca_coms = [10, 0]
cluster_method = ["kmeans", "gmm"]
cluster_sizes = [10, 30]
metricks = ["mahalanobis", "cosine"]
cluster_results = []
for cluster_met, met, clus, pca_com in tqdm(product(cluster_method, metricks, cluster_sizes, pca_coms)):
    oe = OodEvaluator(norm_feats(id_train), norm_feats(id_test), id_train_targ, id_test_targ,
                    num_clusters = clus,
                    pca_com = pca_com,
                    cluster_method = cluster_met,
                    means = None,
                    im2cluster = None,
                    clip = 0,
                    clip_metric = None)

    if met == "mahalanobis" and pca_com == 0:
        oe(norm_feats(ood_test), met, global_cov=True, inv_choice="default", recal=True)
        global_mal_cov = True
    else:
        oe(norm_feats(ood_test), met)
        global_mal_cov = False

    oe.get_scores()
    oe.get_auroc()

    gt_labels = id_train_targ
    pred_labels = oe.in_train_y
    print("homogeneity score")
    hm = metrics.homogeneity_score(gt_labels, pred_labels)
    print("completeness score")
    cmp = metrics.completeness_score(gt_labels, pred_labels)
    print("v measure score, beta = 1")
    vm = metrics.v_measure_score(gt_labels, pred_labels)
    print("adjusted rand score")
    ari = metrics.adjusted_rand_score(gt_labels, pred_labels)
    print("adjusted mutual info score")
    ami = metrics.adjusted_mutual_info_score(gt_labels, pred_labels)

    if clus > 1:
        print("silohuette - cosine")
        sil_cos = metrics.silhouette_score(oe.in_train_X, pred_labels, metric='cosine')
        print("silohuette - mahalanobis")
        #sil_mal = metrics.silhouette_score(oe.in_train_X, pred_labels, metric='mahalanobis')
        sill_mal = 0
        print("davis boulding")
        db_score = metrics.davies_bouldin_score(oe.in_train_X, pred_labels)
        print("calinski harabasz")
        ch_score = metrics.calinski_harabasz_score(oe.in_train_X, pred_labels)
    else:
        sil_cos = 0
        sil_mal = 0
        db_score = 0
        ch_score = 0

    result = {
        "met": met,
        "pca": pca_com,
        "clus": clus,
        "global_mal_cov": global_mal_cov,
        "gmm_results": oe.gmm_results,
        "cluster_method": cluster_met,
        "n_auroc": oe.auroc[0],
        "n_tnr": oe.tnr_at_tpr95[0],
        "e_auroc": oe.auroc[1],
        "e_tnr": oe.tnr_at_tpr95[1],
        "sklearn_auroc": oe.auroc[2],
        "in_train_y":oe.in_train_y,
        "train_pred_label": oe.train_pred_label,
        "test_pred_label": oe.test_pred_label,
        "out_pred_label": oe.out_pred_label,
        "train_pred_score": oe.train_pred_score,
        "test_pred_score": oe.test_pred_score,
        "out_pred_score": oe.out_pred_score,
        "hm": hm,
        "cmp": cmp,
        "vm": vm,
        "ari": ari,
        "ami": ami,
        "sil_cos": sil_cos,
        "sil_mal": sil_mal,
        "db_score": db_score,
        "ch_score": ch_score
    }
    cluster_results.append(result)
    pprint(result)

In [ ]:
gmm_keys = result["gmm_results"].keys()

In [ ]:
new_results = []
for res in cluster_results:
    if res["gmm_results"]:
        for k, v in res["gmm_results"].items():
            res[k] = v
    else:
        for k in gmm_keys:
            res[k] = 0.0
    new_results.append(res)        

In [ ]:
results_df = pd.DataFrame(new_results)

In [ ]:
results_df = pd.read_pickle("/home2/temiloluwa.adeoti/contrastive_ood_detection/cache/cluster_eval_res.pkl")

In [ ]:
results_df

### Extrinsic Analysis


-    homogeniety score
-    complenteness score
-    v measure score
-    adjusted rand score
-    adjusted mutal information


In [ ]:
for cluster_met, met, pca_com in tqdm(product(cluster_method, metricks, pca_coms)):
    met_choice=met
    clus_choice=cluster_met
    pca_choice=pca_com
    _df = results_df[(results_df.met==met_choice) & (results_df.cluster_method==clus_choice) & (results_df.pca==pca_choice)]

    fig = plt.figure(figsize=(12, 10))
    plt.title(f"Distance-metric: {met_choice}; Clustering-algo: {clus_choice};  Pca-components: {pca_choice}")
    plt.plot(_df.hm.values, label="homogeniety")
    plt.plot(_df.ari.values, label="adjusted rand score")
    plt.plot(_df.ami.values, label="adjusted mutual info score")
    plt.plot(_df.cmp.values, label="completeness score")
    plt.plot(_df.vm.values, label="v measure score, beta = 1")
    plt.plot(_df.n_auroc.values/100, label="n_auroc")
    plt.plot(_df.e_auroc.values/100, label="e_auroc")
    if clus_choice == "gmm":
        plt.plot(_df.gmm_default.values, label='gmm_default_auroc')
        plt.plot(_df.gmm_max_prob.values, label='gmm_max_prob_auroc')
        plt.plot(_df.gmm_weighted_max_prob.values, label='gmm_weighted_max_prob_auroc')
        
    plt.xticks(np.arange(len(cluster_sizes)), cluster_sizes)
    plt.yticks(np.linspace(0, 1, 9), np.linspace(0, 1, 9))
    plt.xlabel("Cluster Sizes")
    plt.ylabel("Cluster External Metrics and AUROC")
    plt.grid(which="major")
    plt.legend(loc="upper right")
    plt.savefig(f"../cache/in-{met_choice}-{clus_choice}-{pca_choice}")

### Intrinsic Evaluation 

- silhoutte - euclidean
- silhoutte - cosine
- calinski harabasz score
- davis bouldin score


In [ ]:
for cluster_met, met, pca_com in tqdm(product(cluster_method, metricks, pca_coms)):
    met_choice=met
    clus_choice=cluster_met
    pca_choice=pca_com
    _df = results_df[(results_df.met==met_choice) & (results_df.cluster_method==clus_choice) & (results_df.pca==pca_choice)]

    fig = plt.figure(figsize=(12, 15))
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)
    ax1.title.set_text(f"Distance-metric: {met_choice}; Clustering-algo: {clus_choice};  Pca-components: {pca_choice}")
    ax1.set_ylabel('Calinski Harabasz Score')
    ax2.set_ylabel('Davis bouldin score')
    ax3.set_ylabel('Silhouette Cosine score')
    ax3.set_xlabel('Cluster sizes')

    ax1.plot(_df.db_score.values[1:], label="calinski harabasz score")
    ax2.plot(_df.ch_score.values[1:], label="davis bouldin score")
    ax3.plot(_df.sil_cos.values[1:], label="Silhouette Cosine score")

    ax1.legend()
    ax2.legend()
    ax3.legend()

    a = ax1.set_xticklabels(cluster_sizes)
    a = ax2.set_xticklabels(cluster_sizes)
    a = ax3.set_xticklabels(cluster_sizes)

    a = ax1.grid()
    a = ax2.grid()
    a = ax3.grid()

    plt.savefig(f"../cache/ex-{met_choice}-{clus_choice}-{pca_choice}")

## Clustering Distribution

In [ ]:
clus_dist_df = results_df[['met', 'pca', 'clus', 'cluster_method',
                               'n_auroc', 'n_tnr', 'e_auroc', 'e_tnr', 'sklearn_auroc',
                               'train_pred_label', 'test_pred_label', 'out_pred_label']]
clus_dist_df.head()

In [ ]:
for cluster_met, pca_com, clus in tqdm(product(cluster_method, pca_coms, [10, 30, 50])):
    fil = (clus_dist_df.pca==pca_com)&(clus_dist_df.clus==clus)&(clus_dist_df.cluster_method==cluster_met)
    _df = clus_dist_df.loc[fil, :]
    _df[["met", "n_auroc","n_tnr", "e_auroc", "e_tnr","sklearn_auroc"]].round(2).T.to_excel(f"../cache/dist-{clus}-{pca_com}-{cluster_met}.xlsx")

In [8]:
for cluster_met, pca_com, clus in tqdm(product(cluster_method, pca_coms, cluster_sizes)):
    fil = (clus_dist_df.pca==pca_com)&(clus_dist_df.clus==clus)&(clus_dist_df.cluster_method==cluster_met)
    _df = clus_dist_df.loc[fil, :]
    _df[["n_auroc","n_tnr", "e_auroc", "e_tnr","sklearn_auroc"]].round(2).to_csv(f"../cache/dist-{clus}-{pca_com}-{cluster_met}.csv")
    fig = plt.figure(figsize=(12, 15))
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)

    met = "mahalanobis"
    ax1.title.set_text(f"ID train distribution - {cluster_met};  Pca-components: {pca_com}; Num Clusters: {clus}")
    ax2.title.set_text(f"ID test distribution - {cluster_met};  Pca-components: {pca_com}; Num Clusters: {clus}")
    ax3.title.set_text(f"OOD distribution - {cluster_met};  Pca-components: {pca_com}; Num Clusters: {clus}")

    train_pred = _df[(_df.met==met)].train_pred_label.values[0]
    e = Counter(train_pred)
    cp = cluster_purity(train_pred, id_train_targ)
    clus_n, clus_count = zip(*e.most_common())
    cp = [cp[i] for i in clus_n]
    ax1.bar(np.arange(len(clus_n)), clus_count)
    ax1.bar(np.arange(len(cp)), [cp[i]*clus_count[i] for i in range(len(clus_n))], label="purity level")
    ax1.set_xticks(np.arange(len(clus_n)))
    ax1.set_xticklabels(clus_n, rotation=90)
    ax1.set_ylabel('Counter per Cluster')
    ax1.legend()

    e = Counter(_df[(_df.met==met)].test_pred_label.values[0])
    e = {k:e[k] for k in clus_n}
    ax2.bar(np.arange(len(clus_n)), e.values())
    ax2.set_xticks(np.arange(len(clus_n)))
    ax2.set_xticklabels(e.keys(), rotation=90)
    ax2.set_ylabel('Counter per Cluster')

    e = Counter(_df[(_df.met==met)].out_pred_label.values[0])
    e = {k:e[k] for k in clus_n}
    ax3.bar(np.arange(len(clus_n)), e.values())
    ax3.set_xticks(np.arange(len(clus_n)))
    ax3.set_xticklabels(e.keys(), rotation=90)
    ax3.set_ylabel('Counter per Cluster')
    ax3.set_xlabel('Cluster Number (Sorted by Train ID frequency)')
    plt.savefig(f"../cache/dist-{clus}-{pca_com}-{cluster_met}")

NameError: name 'cluster_method' is not defined

In [4]:
e = pd.DataFrame(pd.read_pickle("/data/temiloluwa.adeoti/fourth_experiments/Imagenet30_clus_768_neg_256/exp_4/checkpoint_199_results/ood_results/metrics_result/contrastive_enc_key_ckpt_199_avg_pool/results.pickle"))

In [5]:
e.keys()

Index(['metric', 'pca components', 'clusters', 'auroc', 'tnr@tpr95',
       'cluster_method', 'gmm results', 'cluster_eval_result',
       'global_mal_cov', 'id_train_y', 'ood', 'idd'],
      dtype='object')

In [7]:
e[['metric', 'pca components', 'clusters', 'auroc', 'tnr@tpr95',
       'cluster_method', 'gmm results', 'global_mal_cov']]

,metric,pca components,clusters,auroc,tnr@tpr95,cluster_method,gmm results,global_mal_cov
0,cosine,0,100,"[63.46378679083081, 66.17615504656851, 66.2747...","[15.791268758526602, 18.34924965893588]",kmeans,{},False
1,cosine,0,100,"[60.30969015591965, 65.57935504277248, 65.6894...","[12.039563437926331, 17.087312414733972]",gmm,"{'gmm_default': 0.6827584622556477, 'gmm_max_p...",False
2,cosine,0,200,"[63.14714225996237, 67.0532956955324, 67.14856...","[15.654843110504773, 26.159618008185536]",kmeans,{},False
3,cosine,0,200,"[64.66498175359038, 67.4522275873307, 67.60821...","[19.133697135061393, 25.443383356070942]",gmm,"{'gmm_default': 0.6728927746254777, 'gmm_max_p...",False
4,cosine,50,100,"[31.13373787295425, 35.88447057636248, 35.8715...","[2.455661664392906, 2.0122783083219646]",kmeans,{},False
5,cosine,50,100,"[29.649385621653302, 34.70707603311065, 34.681...","[2.046384720327422, 1.7394270122783082]",gmm,"{'gmm_default': 0.2567543021298048, 'gmm_max_p...",False
6,cosine,50,200,"[26.237197943569683, 35.306002624353575, 35.26...","[2.762619372442019, 1.6712141882673943]",kmeans,{},False
7,cosine,50,200,"[23.63082488214944, 35.69950231488447, 35.6314...","[2.1828103683492497, 1.7735334242837655]",gmm,"{'gmm_default': 0.26958511812368674, 'gmm_max_...",False
8,mahalanobis,0,100,"[63.730838434442774, 63.53951645047036, 63.535...","[21.964529331514324, 22.544338335607094]",kmeans,{},True
9,mahalanobis,0,100,"[0.3410641200545683, 0.3410641200545683, 67.84...","[0.34106412005457026, 0.34106412005457026]",gmm,"{'gmm_default': 0.6783627915363324, 'gmm_max_p...",True


## Contrastive with pretrained model

In [32]:
ckpt = 59
fn = f"/data/temiloluwa.adeoti/fourth_experiments/CIFAR10_clus_768_neg_256/exp_700/checkpoint_{ckpt}_results/ood_results/metrics_result/contrastive_enc_key_ckpt_{ckpt}_avg_pool/results.pickle"
df = pd.DataFrame(pd.read_pickle(fn))
df[(df.pca==0)&(df.clusters==30)]

,metric,pca,clusters,n_auroc,n_tnr,e_auroc,e_tnr,sklearn_auroc,cluster_method,gmm_default,gmm_max_prob,gmm_weighted_max_prob,global_mal_cov,ood,idd,linear_results
1,cosine,0,30,78.791778,29.45,83.771243,37.33,83.826003,kmeans,None,None,None,False,CIFAR100,CIFAR10,"{'data_set': 'CIFAR10', 'train_accuracy': 94.3..."
6,mahalanobis,0,30,89.510509,56.41,89.185331,54.12,91.126213,kmeans,None,None,None,True,CIFAR100,CIFAR10,"{'data_set': 'CIFAR10', 'train_accuracy': 94.3..."
7,mahalanobis,0,30,48.311551,6.30,0.000336,0.00,51.526456,kmeans,None,None,None,False,CIFAR100,CIFAR10,"{'data_set': 'CIFAR10', 'train_accuracy': 94.3..."


In [39]:
cifar10_k_root = "/data/temiloluwa.adeoti/fourth_experiments/CIFAR10_clus_768_neg_256/exp_1/checkpoint_199_results/ood_results/metrics_result/contrastive_enc_key_ckpt_199_avg_pool"
imgnet_k_root = "/data/temiloluwa.adeoti/fourth_experiments/Imagenet30_clus_768_neg_256/exp_7/checkpoint_199_results/ood_results/metrics_result/contrastive_enc_key_ckpt_199_avg_pool"

In [76]:
#colms = ["metric",  "ood", "pca", "clusters", "e_auroc", "sklearn_auroc", "linear_results", "global_mal_cov"]
cdfk = pd.DataFrame(pd.read_pickle(os.path.join(cifar10_k_root, "results.pickle")))
cdfk = cdfk[(cdfk.global_mal_cov==False) & (cdfk.pca == 0)]
cdfk = cdfk[["metric", "pca", "clusters", "n_auroc", "n_tnr", "e_auroc", "e_tnr", "sklearn_auroc"]]
cdfk

,metric,pca,clusters,n_auroc,n_tnr,e_auroc,e_tnr,sklearn_auroc
1,mahalanobis,0,0,62.327785,28.32,62.344450,27.88,76.967037
3,mahalanobis,0,10,58.643977,19.25,58.738681,19.70,74.038044
5,mahalanobis,0,20,46.178256,3.81,0.001916,0.00,47.512818
7,mahalanobis,0,30,46.418464,4.18,0.000068,0.00,46.441373
9,mahalanobis,0,40,46.385698,2.83,0.000450,0.00,45.959307
11,mahalanobis,0,50,46.775432,3.91,0.000719,0.00,46.912550
48,cosine,0,0,65.585174,11.77,64.982332,4.66,65.169849
49,cosine,0,10,60.920344,8.69,62.992813,6.79,63.078814
50,cosine,0,20,65.550943,10.78,66.494544,7.04,66.641570
51,cosine,0,30,66.704524,12.03,69.009767,11.15,69.297583


In [77]:
cdfn = pd.DataFrame(pd.read_pickle(os.path.join(cifar10_k_root, "new_auroc.pickle")))
cdfn = cdfn.iloc[(cdfk.index)][["distance", "global_train", "global_test", "cluster_train", "cluster_test"]]
cdfn = cdfn.apply(lambda x:x*100)
cdfn

,distance,global_train,global_test,cluster_train,cluster_test
1,76.967037,74.514195,76.962037,73.107750,74.640520
3,74.038044,71.150239,74.033044,69.767129,73.003492
5,47.512818,47.512684,47.507822,47.005604,48.207581
7,46.441373,46.441299,46.436374,46.203660,46.105283
9,45.959307,45.959315,45.954307,46.286522,46.148033
11,NaN,NaN,NaN,NaN,NaN
48,34.830151,34.830202,34.825182,34.963490,35.381765
49,36.921185,36.921209,36.916259,37.101271,37.119303
50,33.358429,33.358400,33.353494,34.275101,34.227657
51,30.702418,30.702386,30.697467,32.427960,32.369380


In [79]:
df = pd.concat([cdfk, cdfn], axis=1)
df = df.round(2)
df[df.metric=="cosine"].T.to_excel("../cache/cif_0_cos.xlsx")

In [113]:
gl_au = []
sk_au = []
m = "mahalanobis"
for ckpt in np.arange(19, 199 + 20, 20):
    ckpt = f"/data/temiloluwa.adeoti/fourth_experiments/CIFAR10_clus_768_neg_256/exp_700/checkpoint_{ckpt}_results/ood_results/metrics_result/contrastive_enc_key_ckpt_{ckpt}_avg_pool/results.pickle"
    df = pd.read_pickle(ckpt)
    dfe = pd.DataFrame(df)
    df = dfe[(dfe.global_mal_cov == False) & (dfe.metric==m)]
    sk_au.append(df.sklearn_auroc)
    df = dfe[(dfe.global_mal_cov == True)]
    gl_au.append(df.sklearn_auroc)

In [126]:
sk_au

[2    88.137554
 Name: sklearn_auroc, dtype: float64,
 2    91.150438
 Name: sklearn_auroc, dtype: float64,
 2    91.630548
 Name: sklearn_auroc, dtype: float64,
 2    91.503116
 Name: sklearn_auroc, dtype: float64,
 2    91.527616
 Name: sklearn_auroc, dtype: float64,
 2    91.20776
 Name: sklearn_auroc, dtype: float64,
 2    90.954693
 Name: sklearn_auroc, dtype: float64,
 2    90.68901
 Name: sklearn_auroc, dtype: float64,
 2    90.701558
 Name: sklearn_auroc, dtype: float64,
 2    90.601291
 Name: sklearn_auroc, dtype: float64]

In [199]:
df = pd.DataFrame(pd.read_pickle("/data/temiloluwa.adeoti/fourth_experiments/CIFAR100_clus_768_neg_256/exp_7020/checkpoint_99_results/ood_results/metrics_result/contrastive_enc_key_ckpt_99_avg_pool/results.pickle"))

In [200]:
df = df[df.global_mal_cov==False]

In [201]:
df = df.rename(columns={"metric":"metric", "pca": "pca", "clusters": "clusters", \
                   "ood": "ood","e_auroc": "cluster auroc", "sklearn_auroc": "global auroc"})
df = df.round(2)
df

,metric,pca,clusters,n_auroc,n_tnr,cluster auroc,e_tnr,global auroc,cluster_method,gmm_default,gmm_max_prob,gmm_weighted_max_prob,global_mal_cov,ood,idd,linear_results
0,cosine,0,0,61.16,10.50,60.21,8.33,60.28,kmeans,None,None,None,False,CIFAR10,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."
1,cosine,0,30,55.66,5.46,55.61,6.38,55.67,kmeans,None,None,None,False,CIFAR10,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."
3,mahalanobis,0,0,46.54,6.86,0.00,0.00,51.09,kmeans,None,None,None,False,CIFAR10,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."
5,mahalanobis,0,30,19.35,8.04,19.37,8.15,56.33,kmeans,None,None,None,False,CIFAR10,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."
6,cosine,0,0,60.54,10.50,55.66,8.33,55.72,kmeans,None,None,None,False,SVHN,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."
7,cosine,0,30,34.16,0.85,40.56,3.80,40.53,kmeans,None,None,None,False,SVHN,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."
9,mahalanobis,0,0,44.83,0.00,0.00,0.00,48.80,kmeans,None,None,None,False,SVHN,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."
11,mahalanobis,0,30,20.38,20.61,20.40,20.61,57.94,kmeans,None,None,None,False,SVHN,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."
12,cosine,0,0,72.65,26.95,73.12,20.18,73.22,kmeans,None,None,None,False,LSUNResize,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."
13,cosine,0,30,66.39,17.65,70.65,15.13,70.69,kmeans,None,None,None,False,LSUNResize,CIFAR100,"{'data_set': 'CIFAR100', 'train_accuracy': 80...."


In [198]:
df[df.metric=="mahalanobis"].to_excel("../cache/mal_cifar.xlsx")

In [206]:
df = pd.DataFrame(pd.read_pickle("/data/temiloluwa.adeoti/fourth_experiments/CIFAR10_clus_768_neg_256/exp_7000/checkpoint_99_results/ood_results/metrics_result/contrastive_enc_key_ckpt_99_avg_pool/new_auroc.pickle"))
df[df.ood=="CIFAR100"]

,distance,global_train,global_test,cluster_train,cluster_test,metric,pca,clusters,cluster_method,ood,idd
0,0.221946,0.221946,0.221896,0.215188,0.214657,cosine,0,0,kmeans,CIFAR100,CIFAR10
1,0.247559,0.247559,0.247510,0.274722,0.275248,cosine,0,30,kmeans,CIFAR100,CIFAR10
2,0.760086,0.760086,0.760036,0.756142,0.755931,mahalanobis,0,0,kmeans,CIFAR100,CIFAR10
3,0.796223,0.796183,0.796173,0.795872,0.774813,mahalanobis,0,0,kmeans,CIFAR100,CIFAR10
4,0.759319,0.759320,0.759269,0.741139,0.738439,mahalanobis,0,30,kmeans,CIFAR100,CIFAR10
5,0.742157,0.564953,0.742107,0.547867,0.714435,mahalanobis,0,30,kmeans,CIFAR100,CIFAR10


In [207]:
df = pd.DataFrame(pd.read_pickle("/data/temiloluwa.adeoti/fourth_experiments/CIFAR10_clus_768_neg_256/exp_700/checkpoint_99_results/ood_results/metrics_result/contrastive_enc_key_ckpt_99_avg_pool/new_auroc.pickle"))
df[df.ood=="CIFAR100"]

,distance,global_train,global_test,cluster_train,cluster_test,metric,pca,clusters,cluster_method,ood,idd
0,0.270640,0.270640,0.270591,0.281562,0.277440,cosine,0,0,kmeans,CIFAR100,CIFAR10
1,0.189885,0.189884,0.189835,0.199940,0.197986,cosine,0,30,kmeans,CIFAR100,CIFAR10
2,0.910552,0.910526,0.910502,0.902369,0.900843,mahalanobis,0,0,kmeans,CIFAR100,CIFAR10
3,0.915276,0.801659,0.915226,0.792867,0.885301,mahalanobis,0,0,kmeans,CIFAR100,CIFAR10
4,0.910680,0.910655,0.910630,0.897090,0.893260,mahalanobis,0,30,kmeans,CIFAR100,CIFAR10
